In [2]:
!pip install transformers
!pip install setfit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=7c096a81483e66c78746aa908de1fabb6e73e332f7fbc4a9adfb5724a3891722
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="Job_Email_Data.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [1]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

In [5]:


# Load a SetFit model from Hub
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset['train'],
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=4,
    num_iterations=20, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"text": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.train()

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 2600
  Num epochs = 1
  Total optimization steps = 650
  Total train batch size = 4


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/650 [00:00<?, ?it/s]

In [6]:
trainer.model._save_pretrained("job_email_classification_model")

## INFERENCE

In [7]:
# Download from Hub and run inference
model = SetFitModel.from_pretrained("./job_classification_model")
# Run inference


OSError: Error no file named pytorch_model.bin, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory ./job_classification_model/.

In [8]:
def infer_one_sentence(sentence):
        predtext = [sentence]
        predicted_class = model(predtext)
        return str(predicted_class)

In [9]:
infer_one_sentence("You have been shortlisted for the interview")

'tensor([1])'

In [10]:
infer_one_sentence("We appreciate your interest in SAP and the time you’ve invested in applying for the Software Developer Intern (SAP AI) opening. Unfortunately we will not be moving forward with your application, nevertheless we would like to share additional information with you, so please take a minute to watch this video message.")

'tensor([1])'

In [11]:
infer_one_sentence("please send your NRIC and passport to move on further.")

'tensor([1])'

In [12]:
infer_one_sentence("""thank you for returning my call earlier! J

As shared, we are happy to offer you an internship opportunity with IHiS for the project: NLP modelling & visualisation on clinical textual data

Kindly refer to the details below for the internship details.

Allowance:                  $1800 per month
Period:                        9 Jan 2023 – 26 May 2023 (credit bearing internship)
Office hours:               830am to 6pm (Monday to Thursday) and 830am to 530pm (Friday), 1h lunch break
Location                      1 Maritime Square #12-01 Harbourfront Centre Singapore 099253""")

'tensor([1])'

In [13]:
infer_one_sentence("""How are you? Please buy this Vitamins for extra health benefits?""")

'tensor([0])'

In [15]:
infer_one_sentence("""
Your job alert for machine learning
3 new jobs in Singapore match your preferences.
Singapore University of Technology and Design (SUTD)
Student Intern
Singapore University of Technology and Design (SUTD) · Singapore (On-site)
Synapxe
3 company alumni

Easy Apply
Easy Apply
 Shopee
Analytics Intern - Features & Games, Regional Marketing
Shopee · Singapore (On-site)
gray radar icon
Actively recruiting
 Temasek
Data Science/Analytics Intern, Finance (Financial Management) Jan - Jun/Jul 2024
Temasek · Singapore (Hybrid)
Nanyang Technological University Singapore
1,296 school alumni
See all jobs
""")



'tensor([0])'